# A introduction of torch.autograd

## Background

## Usage in PyTorch

In [2]:
import torch, torchvision

model = torchvision.models.resnet18(pretrained=True)

data = torch.rand(1, 3, 64, 64)

labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/minioat/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 51.2MB/s]


In [3]:
prediction = model(data) # forward pass

In [4]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

In [5]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [6]:
optim.step() # gradient descent

## Differentiation in autograd